<a href="https://colab.research.google.com/github/meharkhanniazi/Farmdar-Projects/blob/main/PotatoDiseaseClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>